In [18]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score

from utils import load_train_data, load_test_data

In [2]:
#data_dir = "../reduced_data/"
data_dir = "../data/"
batch_size = 32
img_height = 224
img_width = 224

In [26]:
train_data, train_labels = load_train_data(f"{data_dir}train/")
test_data, test_labels = load_test_data(f"{data_dir}new_test/")

Found 10266 files belonging to 29 classes.
Found 1601 files belonging to 29 classes.


In [27]:
train_data = tf.keras.applications.resnet50.preprocess_input(train_data)
test_data = tf.keras.applications.resnet50.preprocess_input(test_data)

In [28]:
train_labels = tf.one_hot(train_labels, 29)
test_labels = tf.one_hot(test_labels, 29)

In [29]:
resnet = tf.keras.applications.resnet.ResNet50(include_top = False, weights='imagenet', input_shape=(224, 224, 3))

resnet.trainable = True
fine_tune_at = 100

for layer in resnet.layers[:fine_tune_at]:
    layer.trainable = False

classifier = resnet.output
classifier = tf.keras.layers.GlobalAveragePooling2D()(classifier)
classifier = tf.keras.layers.Dense(29, activation='softmax')(classifier)
model = tf.keras.Model(inputs=resnet.input, outputs=classifier)

loss = tf.keras.losses.CategoricalCrossentropy()
#optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)
metrics = [tf.keras.metrics.categorical_accuracy]
patience = 10
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, verbose=0, mode='auto', restore_best_weights=True)]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [30]:
#hist = model.fit(train_data, train_labels, epochs=30, callbacks=callbacks, validation_data=(test_data, test_labels), batch_size=128)
hist = model.fit(train_data, train_labels, epochs=30, callbacks=callbacks, validation_split=0.2, batch_size=32)

Epoch 1/30
81/81 [==============================] - 24s 260ms/step - loss: 1.2148 - categorical_accuracy: 0.6226 - val_loss: 9.3317 - val_categorical_accuracy: 0.0369
Epoch 2/30
81/81 [==============================] - 18s 227ms/step - loss: 0.4560 - categorical_accuracy: 0.8264 - val_loss: 12.6150 - val_categorical_accuracy: 0.0169
Epoch 3/30
81/81 [==============================] - 18s 227ms/step - loss: 0.4500 - categorical_accuracy: 0.8377 - val_loss: 12.2918 - val_categorical_accuracy: 0.0106
Epoch 4/30
81/81 [==============================] - 18s 227ms/step - loss: 0.4138 - categorical_accuracy: 0.8440 - val_loss: 9.9726 - val_categorical_accuracy: 0.0137
Epoch 5/30
81/81 [==============================] - 18s 229ms/step - loss: 0.4870 - categorical_accuracy: 0.8526 - val_loss: 4.9954 - val_categorical_accuracy: 0.0319
Epoch 6/30
81/81 [==============================] - 18s 227ms/step - loss: 0.4488 - categorical_accuracy: 0.8438 - val_loss: 7.2034 - val_categorical_accuracy: 0.0

In [ ]:
preds = model(test_data)
acc = accuracy_score(np.argmax(test_labels, axis=-1), np.argmax(preds, axis=-1))
print(acc)
#f1_score(np.argmax(test_labels, axis=-1), np.argmax(preds, axis=-1))

0.037476577139287946
